In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_site_dropdown_values = [{"label":"All Sites", "value":"ALL"}]
for site in spacex_df["Launch Site"].unique():
    launch_site_dropdown_values.append({"label":site,"value":site})


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=launch_site_dropdown_values,
                                             placeholder="Select a Launch Site here",
                                             searchable=True, 
                                             value="ALL"),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                               min=0,
                                               max=10000,
                                               step=1000,
                                               marks={0:"0",
                                                     2500:"2500",
                                                     5000:"5000",
                                                     7500:"7500",
                                                     10000:"10000"},
                                               value=[0,10000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id="success-pie-chart", component_property="figure"),
            Input(component_id="site-dropdown", component_property="value"))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        pie_df = spacex_df.loc[spacex_df["Launch Site"] == entered_site]
        fig = px.pie(pie_df, names="class",
        title='Total Success Launches for site '+entered_site)
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output("success-payload-scatter-chart", "figure"),
            [Input("site-dropdown", "value"),
            Input("payload-slider", "value")])

def get_scatter_chart(entered_site, payload):
    if entered_site == "ALL":
        fig2 = px.scatter(spacex_df, x="Payload Mass (kg)", y="class", color="Booster Version Category", title="Correlation between Payload and Success for all Sites")
        return fig2
    else:
        scatter_df = spacex_df.loc[(spacex_df["Payload Mass (kg)"] >= payload[0]) & (spacex_df["Payload Mass (kg)"] <= payload[1]) & (spacex_df["Launch Site"] == entered_site)]
        fig2 = px.scatter(scatter_df, x="Payload Mass (kg)", y="class", color="Booster Version Category", title="Correlation between Payload and Success for site "+ entered_site)
        return fig2
              

# Run the app
if __name__ == '__main__':
    app.run_server()